In [1]:
import sqlite3 as sql
import pandas as pd

In [2]:
conn = sql.connect('pokedex.db')
c = conn.cursor()

def executeScriptsFromFile(filename):
    # Open and read the file as a single buffer
    fd = open(filename, 'r')
    sqlFile = fd.read()
    fd.close()

    # all SQL commands (split on ';')
    sqlCommands = sqlFile.split(';')

    # Execute every command from the input file
    for command in sqlCommands:
        c.execute(command)

In [3]:
executeScriptsFromFile('sql/criacao_tabelas.sql')

executeScriptsFromFile('sql/adc_pokemon.sql')
executeScriptsFromFile('sql/adc_tipos_pokemon.sql')
executeScriptsFromFile('sql/adc_ataques.sql')
executeScriptsFromFile('sql/adc_uso_ataque.sql')
executeScriptsFromFile('sql/adc_habilidades.sql')
executeScriptsFromFile('sql/adc_uso_habilidades.sql')
executeScriptsFromFile('sql/adc_uso_pokemon.sql')
executeScriptsFromFile('sql/add_evolucoes.sql')
executeScriptsFromFile('sql/adc_aliados.sql')

In [4]:
# Consulta 1.1 (Operações de Seleção e Projeção simples)
# Lista os conteúdos da tabela Pokémon (Nome do pokemon, junto com
# sua descrição e número)
consulta1_1 = pd.read_sql_query("SELECT *"
                                + "FROM Pokemon", conn)
consulta1_1

,id_pokemon,nome,descricao
0,1,Bulbasaur,A strange seed was planted on its back at birt...
1,2,Ivysaur,"When the bulb on its back grows large, it appe..."
2,3,Venusaur,The plant blooms when it is absorbing solar en...
3,4,Charmander,"Obviously prefers hot places. When it rains, s..."
4,5,Charmeleon,"When it swings its burning tail, it elevates t..."
5,6,Charizard,Spits fire that is hot enough to melt boulders...
6,7,Squirtle,"After birth, its back swells and hardens into ..."
7,8,Wartortle,Often hides in water to stalk unwary prey. For...
8,9,Blastoise,A brutal POKéMON with pressurized water jets o...
9,10,Caterpie,Its short feet are tipped with suction pads th...


In [5]:
# Consulta 1.2 (Operações de Seleção e Projeção simples)
# Lista os conteúdos da tabela Ataques (Nomes dos ataques, junto
# da sua descrição, tipo e número de identificação)
consulta1_2 = pd.read_sql_query("SELECT *"
                                + "FROM Ataque", conn)
consulta1_2

,id_ataque,nome,descricao,tipo
0,1,Pound,Pounds with fore­ legs or tail.,normal
1,2,Karate-chop,Has a high criti­ cal hit ratio.,lutador
2,3,Double-slap,Repeatedly slaps 2-5 times.,normal
3,4,Comet-punch,Repeatedly punches 2-5 times.,normal
4,5,Mega-punch,A powerful punch thrown very hard.,normal
5,6,Pay-day,Throws coins. Gets them back later.,normal
6,7,Fire-punch,A fiery punch. May cause a burn.,fogo
7,8,Ice-punch,An icy punch. May cause freezing.,gelo
8,9,Thunder-punch,An electric punch. It may paralyze.,eletrico
9,10,Scratch,Scratches with sharp claws.,normal


In [6]:
# Consulta 2.1 (Junção de 2 Relações)
# Lista os Pokémons que são do tipo fogo e agua simultaneamente
consulta2_1 = pd.read_sql_query("SELECT nome"
                                + " FROM Pokemon NATURAL JOIN TipoPokemon"
                                + " WHERE tipo = 'fogo' AND nome IN(SELECT nome"
                                + " FROM Pokemon NATURAL JOIN TipoPokemon"
                                + " WHERE tipo = 'agua')", conn)
consulta2_1

,nome
0,Volcanion


In [7]:
# Consulta 2.2 (Junção de 2 Relações)
# Lista os Pokémons que não possuem evolução
# e que não são evolução de outro pokemon.
# A junção de 2 Relações se encontra no 2º SELECT 
consulta2_2 = pd.read_sql_query("SELECT nome"
                                + " FROM Pokemon"
                                + " WHERE nome NOT IN(SELECT DISTINCT nome"
                                + " FROM Pokemon JOIN Evolucao ON id_de = id_pokemon OR id_para = id_pokemon)"
                                , conn)
consulta2_2

,nome
0,Farfetchd
1,Kangaskhan
2,Pinsir
3,Tauros
4,Lapras
5,Ditto
6,Aerodactyl
7,Articuno
8,Zapdos
9,Moltres


In [8]:
# Consulta 2.3 (Junção de 2 Relações)
# Lista os Pokémons com uso maior ou igual a 25%
# mostrando seu nome, o ano, mes e a porcentagem
# com os anos em ordem decrescente e os meses de cada ano de forma crescente
# (dados de porcentagem de uso do nível OU de batalha da Smogon)
consulta2_3 = pd.read_sql_query("SELECT nome, ano, mes, porcentagem"
                                + " FROM Pokemon NATURAL JOIN UsoPokemon"
                                + " WHERE porcentagem >= 0.25"
                                + " ORDER BY ano DESC, mes ASC", conn)
consulta2_3

,nome,ano,mes,porcentagem
0,Landorus-incarnate,2019,1,0.444664
1,Magearna,2019,1,0.263640
2,Landorus-incarnate,2019,2,0.446335
3,Magearna,2019,2,0.280846
4,Landorus-incarnate,2019,3,0.459280
5,Magearna,2019,3,0.288188
6,Landorus-incarnate,2019,4,0.419560
7,Magearna,2019,4,0.279323
8,Landorus-incarnate,2018,1,0.490695
9,Landorus-incarnate,2018,2,0.463977


In [9]:
# Consulta 3.1 (Junção de 3 ou mais Relações)
# Lista de Habilidades por Pokemon, em ordem de uso (mais usadas às menos usadas)
# (dados de porcentagem de uso do nível OU de batalha da Smogon)
consulta3_1 = pd.read_sql_query('SELECT p.nome, h.nome, h.descricao'
                                + ' FROM (Habilidade AS h NATURAL JOIN UsoHabilidade AS uh)'
                                + ' JOIN Pokemon AS p ON p.id_pokemon = uh.id_pokemon'
                                + ' GROUP BY p.id_pokemon, h.id_habilidade'
                                + ' ORDER BY p.id_pokemon ASC, uh.porcentagem DESC', conn)
consulta3_1

,nome,nome,descricao
0,Venusaur,Thick Fat,The Pokémon is protected by a layer of thick f...
1,Venusaur,Chlorophyll,Boosts the Pokémon’s Speed stat in harsh sunli...
2,Venusaur,Overgrow,Powers up Grass-type moves when the Pokémon’s ...
3,Charizard,Drought,Turns the sunlight harsh when the Pokémon ente...
4,Charizard,Tough Claws,Powers up moves that make direct contact.
5,Charizard,Solar Power,"Boosts the Sp. Atk stat in harsh sunlight, but..."
6,Charizard,Blaze,Powers up Fire-type moves when the Pokémon’s H...
7,Charizard,Illuminate,Raises the likelihood of meeting wild Pokémon ...
8,Wartortle,Rain Dish,The Pokémon gradually regains HP in rain.
9,Wartortle,Torrent,Powers up Water-type moves when the Pokémon’s ...


In [10]:
# Consulta 3.2 (Junção de 3 ou mais Relações)
# Lista de Ataques por Pokemon, em ordem de uso (mais usadas às menos usadas)
# (dados de porcentagem de uso do nível OU de batalha da Smogon)
consulta3_2 = pd.read_sql_query('SELECT p.nome, a.nome, a.descricao'
                                + ' FROM (Ataque AS a NATURAL JOIN UsoAtaque AS ua)'
                                + ' JOIN Pokemon AS p ON p.id_pokemon = ua.id_pokemon'
                                + ' GROUP BY p.id_pokemon, a.id_ataque'
                                + ' ORDER BY p.id_pokemon ASC, ua.porcentagem DESC', conn)
consulta3_2

,nome,nome,descricao
0,Venusaur,Synthesis,Restores HP (varies by time).
1,Venusaur,Growth,Raises the SPCL. ATK rating.
2,Venusaur,Hidden-power,The power varies with the POKéMON.
3,Venusaur,Sludge-bomb,An attack that may poison the foe.
4,Venusaur,Giga-drain,Steals 1/2 of the damage inflicted.
5,Venusaur,Sleep-powder,May cause the foe to fall asleep.
6,Venusaur,Earthquake,Tough but useless vs. flying foes.
7,Venusaur,Leech-seed,Steals HP from the foe on every turn.
8,Venusaur,Solar-beam,1st turn: Prepare 2nd turn: Attack
9,Venusaur,Leaf-storm,A storm of sharp leaves is whipped up. The att...


In [11]:
# Consulta 3.3 (Junção de 3 ou mais Relações)
# Lista de Pokemons que, em um dado mês e ano, já foram mais utilizados em batalha
# do que suas evoluções, e qual evolução à que se refere
# (dados de porcentagem de uso do nível OU de batalha da Smogon)
consulta3_3 = pd.read_sql_query('SELECT DISTINCT p1.nome, p2.nome'
                                + ' FROM (((Pokemon NATURAL JOIN UsoPokemon) AS p1 JOIN Evolucao as ev'
                                + ' ON p1.id_pokemon = ev.id_de) JOIN (Pokemon NATURAL JOIN UsoPokemon) as p2'
                                + ' ON p2.id_pokemon = ev.id_para)'
                                + ' WHERE p1.ano = p2.ano AND p1.mes = p2.mes'
                                + ' AND p1.porcentagem > p2.porcentagem', conn)
consulta3_3

,nome,nome
0,Bulbasaur,Ivysaur
1,Charmander,Charmeleon
2,Squirtle,Wartortle
3,Caterpie,Metapod
4,Weedle,Kakuna
5,Weedle,Beedrill
6,Kakuna,Beedrill
7,Pidgey,Pidgeotto
8,Pidgey,Pidgeot
9,Rattata,Raticate


In [12]:
# Consulta 4.1 (Funções de Agregação):
# Retorna o Pokemon mais utilizado em batalhas durante todo o tempo de 2014 a 2019
# (do nível OU de batalha da Smogon)
consulta4_1 = pd.read_sql_query('SELECT nome'
                         + ' FROM Pokemon NATURAL JOIN UsoPokemon'
                         + ' WHERE porcentagem = '
                         + '(SELECT MAX(porcentagem) FROM UsoPokemon)', conn)
consulta4_1

,nome
0,Landorus-incarnate


In [13]:
# Consulta 4.1 (Funções de Agregação):
# Retorna uma lista com os anos e os tipos utilizados em batalhas,
# agrupados por ano (decrescente) por ordem de uso (do mais utilizado ao menos utilizado daquele ano)
# (do nível OU de batalha da Smogon)
consulta4_2 = pd.read_sql_query('SELECT ano, tipo'
                                + ' FROM UsoPokemon NATURAL JOIN Pokemon NATURAL JOIN TipoPokemon'
                                + ' GROUP BY ano, tipo'
                                + ' ORDER BY ano DESC, AVG(porcentagem) DESC', conn)
consulta4_2

,ano,tipo
0,2019,metal
1,2019,fada
2,2019,terra
3,2019,voador
4,2019,noturno
5,2019,agua
6,2019,dragao
7,2019,eletrico
8,2019,grama
9,2019,psiquico
